## São Paulo price analysis per borough

In [22]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0 --yes 
import folium
import json
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans

print("Done importing everything!")

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Done importing everything!


In [31]:
#Do the request to retrieve the week page with Sao Paulo boroughs
url_sp_boroughs = "https://en.wikipedia.org/wiki/Subdivisions_of_São_Paulo"
page_text = requests.get(url_sp_boroughs).text

#From the whole html of the wiki page, let's separate the interesting table
page_html = BeautifulSoup(page_text,"lxml")
boroughs_table_html = page_html.find("table",class_="toccolours")

In [32]:
#Create the dataframe from the hmlt and check what is looks like
sp_boroughs = pd.read_html(str(boroughs_table_html))[0]
sp_boroughs

,Regional prefecture,Regional prefecture.1,Regional prefecture.2,Regional prefecture.3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,1.0,Aricanduva,NaN,17.0,M'Boi Mirim
2,2.0,Butantã,NaN,18.0,Mooca
3,3.0,Campo Limpo,NaN,19.0,Parelheiros
4,4.0,Capela do Socorro,NaN,20.0,Penha
5,5.0,Casa Verde,NaN,21.0,Perus
6,6.0,Cidade Ademar,NaN,22.0,Pinheiros
7,7.0,Cidade Tiradentes,NaN,23.0,Pirituba-Jaraguá
8,8.0,Ermelino Matarazzo,NaN,24.0,Santana-Tucuruvi
9,9.0,Freguesia-Brasilândia,NaN,25.0,Santo Amaro


In [33]:
#Drop the useless columns full of NaN
sp_boroughs.drop(["Regional prefecture","Regional prefecture.2","Regional prefecture.3"],axis=1,inplace=True)

#Drop the useless rows full of NaN
sp_boroughs.dropna(axis=0,inplace=True)

#The dataframe now has two interesting columns, we want to put all info in one single column

#Create an auxiliary dataframe with the last column
sp_boroughs_aux = sp_boroughs[["Unnamed: 4"]]
sp_boroughs_aux.columns = ["Regional prefecture.1"]

#Concatenate the original dataframe with the auxiliary one: put the info in the last column under in the 1st column
sp_boroughs = pd.concat([sp_boroughs,sp_boroughs_aux],ignore_index=True,axis=0,sort=False)

#The last column is now useless, drop it
sp_boroughs.drop(["Unnamed: 4"],axis=1,inplace=True)

#Check out the result
sp_boroughs

,Regional prefecture.1
0,Aricanduva
1,Butantã
2,Campo Limpo
3,Capela do Socorro
4,Casa Verde
5,Cidade Ademar
6,Cidade Tiradentes
7,Ermelino Matarazzo
8,Freguesia-Brasilândia
9,Guaianases
